# Entraînement d'un CNN avec des images de spectrogrammes

Ce notebook entraîne un réseau de neurones convolutionnel (CNN) en utilisant des images de spectrogrammes des vibrations du moteur. Le jeu de données est divisé en ensembles d'entraînement, de validation et de test. Le modèle entraîné est sauvegardé avec un horodatage et une convention de nommage basée sur les performances.

## Importation des bibliothèques (libraries) nécessaires

Nous allons importer les bibliothèques nécessaires pour le chargement des données, le prétraitement et la construction du modèle CNN.

In [26]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Chargement et prétraitement des données

Nous allons charger les images de spectrogrammes depuis le répertoire `data/05_cnn_input`, les prétraiter, puis les diviser en ensembles d'entraînement, de validation et de test.

In [ ]:
data_dir = "./../../data/05_cnn_input"

print("Contenu du dossier 05_cnn_input :", os.listdir(data_dir))

image_paths = []
labels = []

for label, category in enumerate(['balourd', 'sain']):
    category_dir = os.path.join(data_dir, category)
    if not os.path.exists(category_dir):
        print(f"Directory does not exist: {category_dir}")
        continue
    print(f"Checking directory: {category_dir}")
    for root, _, files in os.walk(category_dir):
        print(f"Found {len(files)} files in {root}")
        for file in files:
            if file.endswith(".png"):
                image_paths.append(os.path.join(root, file))
                labels.append(label)

print(f"Total directories checked: {len(['balourd', 'sain'])}")
print(f"Total image paths collected: {len(image_paths)}")
print("Sample image paths:", image_paths[:5])
print("Labels distribution:", {label: labels.count(label) for label in set(labels)})

image_paths = np.array(image_paths)
labels = np.array(labels)

Contenu du dossier 05_cnn_input : ['balourd', 'sain', 'test', 'train', 'val']
Checking directory: ./../../data/05_cnn_input/balourd
Found 246 files in ./../../data/05_cnn_input/balourd
Checking directory: ./../../data/05_cnn_input/sain
Found 551 files in ./../../data/05_cnn_input/sain
Total directories checked: 2
Total image paths collected: 797
Sample image paths: ['./../../data/05_cnn_input/balourd/spec_rgb_0551.png', './../../data/05_cnn_input/balourd/spec_rgb_0552.png', './../../data/05_cnn_input/balourd/spec_rgb_0553.png', './../../data/05_cnn_input/balourd/spec_rgb_0554.png', './../../data/05_cnn_input/balourd/spec_rgb_0555.png']
Labels distribution: {0: 246, 1: 551}


In [28]:
# Appliquer le prétraitement pour fusionner les spectrogrammes
images = []
filtered_labels = []  # Nouvelle liste pour les étiquettes correspondantes

for i in range(0, len(image_paths), 3):  # Supposer que les fichiers sont triés par moment
    if i + 2 < len(image_paths):
        merged_image = merge_spectrograms(image_paths[i], image_paths[i + 1], image_paths[i + 2])
        if merged_image is not None:
            images.append(merged_image)
            filtered_labels.append(labels[i])  # Ajouter l'étiquette correspondante

images = np.array(images)
filtered_labels = np.array(filtered_labels)  # Convertir en tableau numpy

# Débogage : Vérifier les images traitées
print(f"Nombre total d'images valides traitées : {len(images)}")
print(f"Nombre total d'étiquettes correspondantes : {len(filtered_labels)}")
if len(images) > 0:
    print("Dimensions de la première image :", images[0].shape)

# Diviser le jeu de données en ensembles d'entraînement, de validation et de test
if len(images) == 0:
    raise ValueError("Aucune image valide n'a été traitée. Veuillez vérifier le jeu de données.")

X_train, X_temp, y_train, y_temp = train_test_split(images, filtered_labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Débogage : Afficher les répartitions des ensembles de données
print(f"Ensemble d'entraînement : {len(X_train)} échantillons")
print(f"Ensemble de validation : {len(X_val)} échantillons")
print(f"Ensemble de test : {len(X_test)} échantillons")

Nombre total d'images valides traitées : 265
Nombre total d'étiquettes correspondantes : 265
Dimensions de la première image : (129, 101, 3)
Ensemble d'entraînement : 159 échantillons
Ensemble de validation : 53 échantillons
Ensemble de test : 53 échantillons


## Construction du modèle CNN

Nous allons définir une architecture de réseau de neurones convolutionnel (CNN) pour traiter les images de spectrogrammes.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Définir le modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(129, 101, 3)),  # Updated input shape
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Afficher le résumé du modèle
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 127, 99, 32)       896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 63, 49, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 61, 47, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 30, 23, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 28, 21, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 14, 10, 128)      

## Entraînement du modèle et sauvegarde

Nous allons compiler le modèle, l'entraîner en utilisant les ensembles d'entraînement et de validation, puis enregistrer le modèle entraîné avec une convention de nommage basée sur les performances.

In [ ]:
from datetime import datetime

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

# Générer un nom de fichier avec un timestamp et les performances
def generate_model_name(history, test_accuracy):
    val_accuracy = max(history.history['val_accuracy']) * 100
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return f"{timestamp}_validation_res_{val_accuracy:.2f}_test_set_{test_accuracy:.2f}.h5"

# Sauvegarder le modèle
model.save(generate_model_name(history, 0))

Epoch 1/10
5/5 [==============================] - 2s 317ms/step - loss: 110.9671 - accuracy: 0.5660 - val_loss: 5.0228 - val_accuracy: 0.6415
Epoch 2/10
5/5 [==============================] - 0s 18ms/step - loss: 4.0660 - accuracy: 0.5975 - val_loss: 0.7262 - val_accuracy: 0.6415
Epoch 3/10
5/5 [==============================] - 0s 17ms/step - loss: 0.9331 - accuracy: 0.6038 - val_loss: 0.6406 - val_accuracy: 0.6415
Epoch 4/10
5/5 [==============================] - 0s 19ms/step - loss: 1.3242 - accuracy: 0.5849 - val_loss: 0.6859 - val_accuracy: 0.6226
Epoch 5/10
5/5 [==============================] - 0s 18ms/step - loss: 0.6553 - accuracy: 0.6855 - val_loss: 0.6515 - val_accuracy: 0.6415
Epoch 6/10
5/5 [==============================] - 0s 18ms/step - loss: 0.6794 - accuracy: 0.6730 - val_loss: 0.6541 - val_accuracy: 0.6415
Epoch 7/10
5/5 [==============================] - 0s 19ms/step - loss: 0.6395 - accuracy: 0.6792 - val_loss: 0.6465 - val_accuracy: 0.6415
Epoch 8/10
5/5 [========

## Évaluation du modèle

Nous allons évaluer le modèle entraîné sur l'ensemble de test et calculer la précision sur cet ensemble.

In [ ]:
## Évaluation du modèle
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble de test : {test_accuracy * 100:.2f}%")

# Sauvegarder le modèle avec la précision mise à jour
model.save(generate_model_name(history, test_accuracy * 100))

2/2 [==============================] - 0s 6ms/step - loss: 0.6028 - accuracy: 0.7736
Test Accuracy: 77.36%
